In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("D:/Github")
from note.get_info_TWSE import GetInfoTWSE
import pandas as pd
from tqdm import tqdm
from pprint import pprint

In [2]:
list = pd.read_csv("stock_id_list.csv")
list = list["Code"].to_list()

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

fetcher = GetInfoTWSE()
for id in tqdm(list):
    try:
        df = fetcher.init_df(str(id), "2020-01-01", "2025-09-04")
        df = fetcher.get_stock_price()
        df = fetcher.get_day_trading_info()
        df.to_csv(f"data/{id}.csv")
    except:
        continue

  1%|          | 13/1066 [44:11<57:27:37, 196.45s/it]D:\Github\note\get_info_TWSE.py:184: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  px = pd.concat(all_data, ignore_index = True)
  2%|▏         | 18/1066 [59:51<55:33:22, 190.84s/it]D:\Github\note\get_info_TWSE.py:184: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  px = pd.concat(all_data, ignore_index = True)
  2%|▏         | 19/1066 [1:02:54<54:52:13, 188.67s/it]D:\Github\note\get_info_TWSE.py:184: FutureWarning: The behavior of DataFrame con